# Rosenkranz-Schlüssel: Q-Simulation für Verkehrssicherheit
Interaktive Optimierung von Risikofaktoren mit Sliders. Basierend auf früherem Code, fixed und erweitert.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import torch  # Falls ML-Elemente später hinzukommen (aus requirements.txt)

In [ ]:
# Fixed Funktionen aus dem alten Code
def skalenvariant_reichweite(risiko):
    return np.sqrt(1 / risiko)  # Proxy für Reichweite in Verkehrs-Skalen

def uberlebens_konstante(Q, risiko_factor, K=1.0):
    return K * (Q / risiko_factor)  # Erweiterte Konstante

def rosenkranz_approximation(Q_start, iterations, unsicherheit=0.1):
    Q_approx = Q_start
    for i in range(1, iterations + 1):
        Q_approx += ((-1)**(i+1) * (unsicherheit**i / i))  # Korrigierte alternierende Serie für bessere Konvergenz
    return Q_approx

In [ ]:
# Interaktive Sim mit Sliders (Verkehrs-Params: Dichte als Risiko, Geschwindigkeit als Skala)
@interact(
    samples=IntSlider(min=50, max=500, step=50, value=100, description='Samples:'),
    min_risiko=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05, description='Min Dichte/Risiko:'),
    max_risiko=FloatSlider(min=0.3, max=1.0, step=0.05, value=0.5, description='Max Dichte/Risiko:'),
    iterations=IntSlider(min=5, max=50, step=5, value=10, description='Q-Iterationen:'),
    unsicherheit=FloatSlider(min=0.05, max=0.5, step=0.05, value=0.1, description='Unsicherheit (z.B. Nebel):')
)
def simulate_verkehr_schluessel(samples, min_risiko, max_risiko, iterations, unsicherheit):
    scale = np.linspace(1, 100, samples)  # Skalen: z.B. Geschwindigkeit km/h
    risikofaktor = np.random.uniform(min_risiko, max_risiko, samples)  # Risiko: z.B. Verkehrsdichte
    Q_start = 1.0  # Initialer Q-Wert
    Q_werte = []
    for s, risiko in zip(scale, risikofaktor):
        reichweite = skalenvariant_reichweite(risiko)
        Q_basis = uberlebens_konstante(Q_start, risiko)
        Q_optim = rosenkranz_approximation(Q_basis, iterations, unsicherheit)
        Q_werte.append(Q_optim * s / 100)  # Skaliere mit Geschwindigkeit für Realismus
    
    plt.figure(figsize=(12, 6))
    plt.plot(scale, Q_werte, label='Optimierter Q-Schlüssel (Sicherheit)')
    plt.xlabel('Geschwindigkeit/Skala (km/h)')
    plt.ylabel('Q-Wert (Überlebenswahrscheinlichkeit)')
    plt.title('Rosenkranz-Schlüssel: Verkehrsrisiko-Optimierung')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    mean_Q = np.mean(Q_werte)
    print(f'Durchschnittlicher Q-Schlüssel: {mean_Q:.2f}')
    if mean_Q > 10:
        print('Schlüssel unlocked: Optimale Ampel/Route freigeschaltet – sichere Fahrt!')
    else:
        print('Schlüssel locked: Risiko hoch – reduziere Dichte oder Iterationen.')